<a href="https://colab.research.google.com/github/BrauuHdzM/Fine-Tuning-LLaMA3/blob/master/Fine_Tuning_LLaMA_3_1_para_Noticias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

[NEW] Llama-3.1 8b, 70b & 405b are trained on a crazy 15 trillion tokens with 128K long context lengths!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
* [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
* [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Llama patching release 2024.8
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [ ]:
!python -m spacy download es_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 1.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
nlp = spacy.load("es_core_news_lg")

In [ ]:
def extract_first_loc(text):
    # Procesa el texto con spaCy para encontrar entidades nombradas
    doc = nlp(text)

    # Busca la primera entidad de tipo LOC
    first_loc = next((ent.text for ent in doc.ents if ent.label_ == "LOC"), None)

    return first_loc

In [ ]:
alpaca_prompt = """Tu tarea es escribir artículos de noticia que contengan siempre una fecha, un lugar y un acontecimiento. No puedes inventar información que no se te da, utiliza lenguaje formal.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = "Crea un artículo de noticias con esta información:"
    inputs = []
    outputs = []
    for contenido, resumen, fecha in zip(examples["Contenido"], examples["resumen"], examples["Fecha"]):
        location = extract_first_loc(contenido)
        if location:
            input_text = resumen + ". " + "Fecha: " + fecha + ". Lugar: " + location
            output_text = "El " + fecha +  ", en " + location + ". " + contenido
        else:
            input_text = resumen + ". " + "Fecha: " + fecha
            output_text = "El " + fecha + ". " + contenido
        print(input_text)
        print(output_text)
        inputs.append(input_text)
        outputs.append(output_text)
    print(len(inputs))
    print(len(outputs))
    texts = []

    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instructions, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("BrauuHdzM/Noticias-con-resumen", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

La Organización Internacional del Trabajo (OIT) reportó que Gaza ha perdido al menos el 61% de sus empleos desde el inicio de la guerra entre Israel y Hamas, lo que equivale a aproximadamente 182,000 puestos de trabajo. La OIT advirtió que las consecuencias económicas de este conflicto se sentirán durante muchos años. Ruba Jaradat, directora regional de la OIT para los Estados Árabes, calificó los resultados de la evaluación del impacto económico como extremadamente preocupantes.. Fecha: 08 de noviembre del 2023. Lugar: Gaza
El 08 de noviembre del 2023, en Gaza. Gaza ha perdido al menos 61 por ciento de empleos desde que comenzó la guerra entre Israel y Hamas, reportó ayer la Organización Internacional del Trabajo, y advirtió que las consecuencias económicas reverberarán durante “muchos años”. Las bajas estimadas de empleo en el territorio palestino equivalen a 182 mil puestos de trabajo, indicó el ente laboral de la Organización de Naciones Unidas en una evaluación del impacto económi

In [ ]:
print(dataset)

Dataset({
    features: ['Título', 'Vínculo', 'Fecha', 'Contenido', 'longitud_contenido', 'resumen', 'text'],
    num_rows: 100
})


In [ ]:
print(dataset["text"][1])

Tu tarea es escribir artículos de noticia que contengan siempre una fecha, un lugar y un acontecimiento. No puedes inventar información que no se te da, utiliza lenguaje formal.

### Instruction:
Crea un artículo de noticias con esta información:

### Input:
La intensidad del huracán Otis, que alcanzó niveles históricos en la zona, fue complicado de prever, ya que únicamente un modelo de análisis meteorológico global logró anticipar que el huracán podría alcanzar categorías muy peligrosas. Esta información fue proporcionada por Claudia Rojas Serna, profesora del departamento de Ingeniería de Procesos e Hidráulica de la Universidad Autónoma Metropolitana, unidad Iztapalapa.. Fecha: 04 de noviembre del 2023. Lugar: departamento de Ingeniería de Procesos

### Response:
El 04 de noviembre del 2023, en departamento de Ingeniería de Procesos. La intensidad del huracán Otis –lamás alta registrada en toda la historia en esa zona del país– era muy difícil de prever, pues sólo un modelo de análi

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.984 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 100 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.956000
2,2.040900
3,1.972300
4,1.800000
5,1.620900
6,1.599000
7,1.365000
8,1.358400
9,1.167400
10,1.058200


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

524.2611 seconds used for training.
8.74 minutes used for training.
Peak reserved memory = 7.717 GB.
Peak reserved memory for training = 1.733 GB.
Peak reserved memory % of max memory = 52.326 %.
Peak reserved memory for training % of max memory = 11.751 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Crea un artículo de noticias con esta información:", # instruction
        "Melbourne. El español Carlos Sainz (Ferrari), ganador del Gran Premio de Australia de Fórmula Uno, aseguró que confiaba en tener la capacidad de superar al tricampeón mundial y actual líder de la competencia, el neerlandés Max Verstappen (Red Bull), hoy en la tercera prueba de la temporada, e indicó que su triunfo demuestra que nunca hay que darse por vencido. Fecha: 25 de marzo del 2024. Lugar: Melbourne", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Tu tarea es escribir artículos de noticia que contengan siempre una fecha, un lugar y un acontecimiento. No puedes inventar información que no se te da, utiliza lenguaje formal.\n\n### Instruction:\nCrea un artículo de noticias con esta información:\n\n### Input:\nMelbourne. El español Carlos Sainz (Ferrari), ganador del Gran Premio de Australia de Fórmula Uno, aseguró que confiaba en tener la capacidad de superar al tricampeón mundial y actual líder de la competencia, el neerlandés Max Verstappen (Red Bull), hoy en la tercera prueba de la temporada, e indicó que su triunfo demuestra que nunca hay que darse por vencido. Fecha: 25 de marzo del 2024. Lugar: Melbourne\n\n### Response:\nEl 25 de marzo del 2024, en Melbourne. Melbourne. El español Carlos Sainz (Ferrari), ganador ayer del Gran Premio de Australia de Fórmula Uno, aseguró hoy que confiaba en su capacidad para superar al tricampeón mundial']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Crea un artículo de noticias con esta información:", # instruction
        "Melbourne. El español Carlos Sainz (Ferrari), ganador del Gran Premio de Australia de Fórmula Uno, aseguró que confiaba en tener la capacidad de superar al tricampeón mundial y actual líder de la competencia, el neerlandés Max Verstappen (Red Bull), hoy en la tercera prueba de la temporada, e indicó que su triunfo demuestra que nunca hay que darse por vencido. Fecha: 25 de marzo del 2024. Lugar: Melbourne", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Tu tarea es escribir artículos de noticia que contengan siempre una fecha, un lugar y un acontecimiento. No puedes inventar información que no se te da, utiliza lenguaje formal.

### Instruction:
Crea un artículo de noticias con esta información:

### Input:
Melbourne. El español Carlos Sainz (Ferrari), ganador del Gran Premio de Australia de Fórmula Uno, aseguró que confiaba en tener la capacidad de superar al tricampeón mundial y actual líder de la competencia, el neerlandés Max Verstappen (Red Bull), hoy en la tercera prueba de la temporada, e indicó que su triunfo demuestra que nunca hay que darse por vencido. Fecha: 25 de marzo del 2024. Lugar: Melbourne

### Response:
El 25 de marzo del 2024, en Melbourne. Melbourne. El español Carlos Sainz (Ferrari), ganador del Gran Premio de Australia de Fórmula Uno, dijo ayer que confiaba en su capacidad para superar al tricampeón mundial y líder de la competencia, el neerlandés Max Verstappen (Red Bull), en la tercera carrer

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
#model.save_pretrained("lora_model") # Local saving
#tokenizer.save_pretrained("lora_model")
model.push_to_hub("BrauuHdzM/llama3_noticias_espanol", token = "") # Online saving
tokenizer.push_to_hub("BrauuHdzM/llama3_noticias_espanol", token = "") # Online saving

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/BrauuHdzM/llama3_noticias_espanol


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "BrauuHdzM/llama3_noticias_espanol", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

==((====))==  Unsloth: Fast Llama patching release 2024.8
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
alpaca_prompt = """Tu tarea es escribir artículos de noticia que contengan siempre una fecha, un lugar y un acontecimiento. No puedes inventar información que no se te da, utiliza lenguaje formal.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Crea un artículo de noticias con esta información:", # instruction
        "Melbourne. El español Carlos Sainz (Ferrari), ganador del Gran Premio de Australia de Fórmula Uno, aseguró que confiaba en tener la capacidad de superar al tricampeón mundial y actual líder de la competencia, el neerlandés Max Verstappen (Red Bull), hoy en la tercera prueba de la temporada, e indicó que su triunfo demuestra que nunca hay que darse por vencido. Fecha: 25 de marzo del 2024. Lugar: Melbourne", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Tu tarea es escribir artículos de noticia que contengan siempre una fecha, un lugar y un acontecimiento. No puedes inventar información que no se te da, utiliza lenguaje formal.

### Instruction:
Crea un artículo de noticias con esta información:

### Input:
Melbourne. El español Carlos Sainz (Ferrari), ganador del Gran Premio de Australia de Fórmula Uno, aseguró que confiaba en tener la capacidad de superar al tricampeón mundial y actual líder de la competencia, el neerlandés Max Verstappen (Red Bull), hoy en la tercera prueba de la temporada, e indicó que su triunfo demuestra que nunca hay que darse por vencido. Fecha: 25 de marzo del 2024. Lugar: Melbourne

### Response:
El 25 de marzo del 2024, en Melbourne. Melbourne. El español Carlos Sainz (Ferrari), ganador del Gran Premio de Australia de Fórmula Uno, aseguró ayer en Melbourne que confiaba en su capacidad para derrotar al tricampeón del mundo y líder actual de la competencia, el neerlandés Max Verstappen (Red B

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")